In [1]:
%pip install transformers
%pip install torch

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install soundfile


Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install Flask  pyngrok pydub


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install accelerate


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, request, jsonify
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from pydub import AudioSegment
import numpy as np
import os
import torch

# Configure device
device = "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load model and processor
model_id = "openai/whisper-small"  # You can change this to any Whisper model
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# Initialize the transcription pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# Function to convert audio and transcribe it
def mp3_to_array(mp3_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_channels(1).set_frame_rate(16000)  # Mono and 16kHz
    audio_np = np.array(audio.get_array_of_samples())
    return audio_np.astype(np.float32) / 32768.0  # Normalize

# Flask app setup
app = Flask(__name__)

@app.route('/transcribe', methods=['POST'])
def transcribe():
    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files['audio']
    audio_path = os.path.join("Doctor", audio_file.filename)
    audio_file.save(audio_path)
    
    # Convert the file to MP3 if necessary
    mp3_path = audio_path.replace(".m4a", ".mp3").replace(".mp4", ".mp3")
    audio = AudioSegment.from_file(audio_path)
    audio.export(mp3_path, format="mp3")
    if not audio_path.endswith(".mp3"):
        os.remove(audio_path)
    
    # Transcribe the audio
    audio_array = mp3_to_array(mp3_path)
    sample = {"array": audio_array, "sampling_rate": 16000}
    
    try:
        result = pipe(sample)
        transcription_text = result.get("text", "")
        return jsonify({"transcription": transcription_text}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.8:5000
Press CTRL+C to quit


In [ ]:
%pip install flask-cors
